# Web Scraping. Capítulo 2.2
## Selenium
### Esperas

Este código permite inicializar selenium, se algo falla durante a execución, non dubides en pechar o navegador e abrir un novo con este bloque.

In [ ]:
#Instalar gecko
from webdriver_manager.firefox import GeckoDriverManager
GeckoDriverManager().install()

#Abrir un navegador
from selenium import webdriver
from selenium.webdriver.common.by import By
driver = webdriver.Firefox()

### Esperas

#### Espera implícita

Selenium trae de serie un mecanismo que espera a que un elemento se cargue. Se non se carga nun tempo predefinido (timeout) entón para a carga.

Este axuste aplícase de xeito global a cada ruta chamada para toda a sesión sendo o valor por defecto cero, polo tanto, se non atopa o elemento, devolve un erro inmediatamente. Se establecemos esta espera a un número diferente, agardará ese número de segundos antes de devolver un erro. Sen embargo tan pronto localice o elemento, devolverao, co cual esta espera implícita non ten porqué incrementar a duración da sesión (sobre todo si non se producen erros).

Está desaconsellado pola documentación mezclar esperas implícitas e explícitas por ter resultados impredecibles e non desexados.

Esta páxina forza unha espera dun segundo, polo que se non agardamos suficiente (quitamos o implicit wait) fallaría. Podes probar a poñer a 0 (por defecto) o «implicit wait».

In [ ]:
driver.implicitly_wait(2)
driver.get('https://www.selenium.dev/selenium/web/dynamic.html')
driver.find_element(By.ID, "adder").click()
added = driver.find_element(By.ID, "box0")

Agora imos ver que si chamamos un elemento que non exista, tardará dous segundos en lanzar o erro

In [ ]:
driver.implicitly_wait(2)
driver.find_element(By.ID, "adder00").click()

E agora o erro será inmediato:

In [ ]:
driver.implicitly_wait(0)
driver.find_element(By.ID, "adder00").click()

Por qué é importante? Non tódalas páxinas son estáticas, algunhas precisan chamar a unha API para descargar uns datos ou cambian en función da túa interacción coa páxina. Mesmo hai algunhas que traen o típico "spinner" e non che deixan interactuar ata que cargan os datos. Nestes casos pode ser útil esperar por un determinado elemento.

#### Espera explícita

Valídase de xeito contínuo (mediante un bucle) que exista unha determinada condición ou elemento antes de continuar. De non cumplirse esta condición nun tempo determinado (timeout) devolverá un erro.

O comportamento por defecto da clase Wait de Selenium é esperar que exista o obxecto.

É unha das esperas máis empregadas.

In [ ]:
driver.get('https://www.selenium.dev/selenium/web/dynamic.html')
revealed = driver.find_element(By.ID, "revealed")
driver.find_element(By.ID, "reveal").click()

from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.support.wait import WebDriverWait

errors = [NoSuchElementException, ElementNotInteractableException]
wait = WebDriverWait(driver, timeout=2, poll_frequency=.2, ignored_exceptions=errors)
wait.until(lambda d : revealed.send_keys("Displayed") or True)

True

**Outro exemplo**:

Imaxinemos que temos unha roda de carga (spinner loading...) e que ten que acabar para deixarnos premer nos botóns ou para que cargue a información que nos interesa.

- Exemplo: <https://jfsanchez.es/cdn/selenium/carga.html>

O texto que nos interesa está nun p con id "mensaxegatos", pero este p ten un valor incorrecto ata que non desaparece o spinner (ata que non carga).

Debemos esperar a que exista na páxina (se cargue o spinner) e logo a que se volva invisible. Tras volverse invisible poderemos obter os datos que nos interesan.

In [ ]:
from selenium.webdriver.support import expected_conditions as EC

def espera_Oh(driver):
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "page-loader")))
    WebDriverWait(driver, 10).until(EC.invisibility_of_element((By.ID, "page-loader")))


In [ ]:
CARGA_URL = 'https://jfsanchez.es/cdn/selenium/carga.html'
driver.get(CARGA_URL)

espera_Oh(driver)
print(driver.find_element(By.ID, "mensaxegatos").text)

MIAU! GUAU! GOF! CROA! CUAC! OINC! GRUNZ!


### Espera fluída

É un tipo de espera cando precisamos unha determinada precisión ou espera entre as comprobación.

Normalmente é considerada unha versión de espera explícita, simplemente nos deixa controlar o tempo de comprobación (polling time) para ser máis exactos (reducíndoo) ou para reducir a potencia de procesador empregada (aumentándoo).

Cando temos moita carga de traballo, pode ser unha boa idea con determinadas webs.

Esta espera fluída cambia na implementación de Java e Python sendo, alomenos en versións anteriores para Java unha clase diferente.

En Python poderíamos especificar como parámtros o timeout e o poll_frecuency.

    wait=WebDriverWait(driver,timeout=10,poll_frequency=1)
    element=wait.until(element_has_text(...))

### Bibliografía

- <https://www.selenium.dev/>
- <https://github.com/SeleniumHQ>
- <https://selenium-python.readthedocs.io>